In [1]:
import os
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img

from create_mask import *
from utils import *

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', fscale=1.2, ticks=True, grid=False, figsize=(8,6))

# 1 - Data Preparation

## 1.1 - Loading and reshaping

In [6]:
list_panos = ['pano_01', 'pano_02', 'pano_03', 'pano_04']

list_img   = create_list_img(list_panos)
list_masks = create_list_masks(list_panos)

In [7]:
for i in range(4):
    print(list_img[i].shape)

(1440, 2976, 3)
(1440, 2972, 3)
(1440, 2976, 3)
(1440, 2976, 4)


In [8]:
list_img, list_masks = reshape_img_masks(list_img,list_masks)

In [9]:
for i in range(4):
    print(list_img[i].shape)

(1440, 2972)
(1440, 2972)
(1440, 2972)
(1440, 2972)


In [ ]:
# plt.figure(figsize=(30,20))
# plt.imshow(list_img[3])

In [ ]:
# plt.figure(figsize=(30,20))
# plt.imshow(ma.masked_where(list_masks[3], list_img[3]))

In [11]:
i = 0
for mask in list_masks:
    i+=1
    plt.imsave('./masks/mask_0'+str(i)+'.png', mask)

## 1.2 - Prepare sequence data

In [12]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class Panoramics(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
        return x, y

In [13]:
input_dir = "img/"
target_dir = "masks/"
img_size = (1440, 2972)
num_classes = 2
batch_size = 2

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

Number of samples: 4
img/pano_01.png | masks/mask_01.png
img/pano_02.png | masks/mask_02.png
img/pano_03.png | masks/mask_03.png
img/pano_04.png | masks/mask_04.png


In [14]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

# Display input image #7
# display(Image(filename=input_img_paths[2]))

# Display auto-contrast version of corresponding target (per-pixel categories)
# img = PIL.ImageOps.autocontrast(load_img(target_img_paths[2]))
# display(img)

# 2 - Model

## 2.1 - Model Instanciation

In [15]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model




# Build model
model = get_model((1440,2972), 2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1440, 2972,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 720, 1486, 32 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 720, 1486, 32 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 720, 1486, 32 0           batch_normalization[0][0]        
______________________________________________________________________________________________

## 2.2 - Train / Validate split

In [16]:
input_img_paths

['img/pano_01.png', 'img/pano_02.png', 'img/pano_03.png', 'img/pano_04.png']

In [17]:
val_samples = 1
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = Panoramics(
    batch_size, img_size, train_input_img_paths, train_target_img_paths
)
val_gen = Panoramics(batch_size, img_size, val_input_img_paths, val_target_img_paths)

## 2.3 - Train the model

In [16]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint("oxford_segmentation.h5", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
epochs = 2
model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

Train for 1 steps
Epoch 1/2
1/1 [==============================] - 88s 88s/step


KeyboardInterrupt: 

In [ ]:
plt.imshow(list_masks[0]*1)